In [ ]:
'''存放处理不同问答类型的工具函数，核心文件'''

import base64

import time
import json
from client.clientfactory import Clientfactory
from qa.purpose_type import userPurposeType
from pathlib import Path
from ppt_docx.ppt_generation import generate as generate_ppt
from ppt_docx.ppt_content import generate_ppt_content
from ppt_docx.docx_generation import generate_docx_content as generate_docx
from ppt_docx.docx_content import generate_docx_content
from rag import rag_chain
from audio.audio_extract import (
    extract_text,
    extract_language,
    extract_gender,
    get_tts_model_name,
)
from audio.audio_generate import audio_generate
from model.KG.search_service import search
from Internet.Internet_chain import InternetSearchChain
from Graph import GraphDao
from config.config import Config
from qa.purpose_type import userPurposeType
from env import get_env_value


_dao = GraphDao()

In [1]:
from ke_search_service import EntitySearcher
from typing import Callable, List, Dict, Tuple
from config import load_nested_params
from dotenv import load_dotenv, dotenv_values
from Graph import GraphDao
load_dotenv(".env", override=False)
_dao= GraphDao()
def relation_tool(entities: List[Dict] | None) -> str | None:
    if not entities or len(entities) == 0:
        return None

    relationships = set()  # 使用集合来避免重复关系
    relationship_match = []

    searchKey = load_nested_params("model", "graph-entity", "search-key")
    # 遍历每个实体并查询与其他实体的关系
    for entity in entities:
        entity_name = entity[searchKey]
        for k, v in entity.items():
            relationships.add(f"{entity_name} {k}: {v}")

        # 查询每个实体与其他实体的关系a-r-b
        relationship_match.append(_dao.query_relationship_by_name(entity_name))
        
    # 抽取并记录每个实体与其他实体的关系
    for i in range(len(relationship_match)):
        for record in relationship_match[i]:
            # 获取起始节点和结束节点的名称

            start_name = record["r"].start_node[searchKey]
            end_name = record["r"].end_node[searchKey]

            # 获取关系类型
            rel = type(record["r"]).__name__  # 获取关系名称，比如 CAUSES

            # 构建关系字符串并添加到集合，确保不会重复添加
            relationships.add(f"{start_name} {rel} {end_name}")

    # 返回关系集合的内容
    if relationships:
        return "；".join(relationships)
    else:
        return None

def check_entity(question: str) -> List[Dict]:
    code, result = EntitySearcher().search(question)
    if code == 0:
        return result
    else:
        return None

# a=check_entity("苹果公司")
# print(a)
# b=relation_tool(a)
# print(b)

setting environment variables: OrderedDict([('LLM_BASE_URL', 'https://dashscope.aliyuncs.com/compatible-mode/v1'), ('LLM_API_KEY', 'sk-3e06f340717e43f59adf6ba05fcb52b5'), ('MODEL_NAME', 'qwen-turbo'), ('PY_ENVIRONMENT', 'web'), ('IMAGE_GENERATE_API', ''), ('IMAGE_GENERATE_MODEL', 'cogview-3-flash'), ('IMAGE_DESCRIBE_API', ''), ('IMAGE_DESCRIBE_MODEL', 'glm-4v-flash'), ('VIDEO_GENERATE_API', ''), ('VIDEO_GENERATE_MODEL', 'cogvideox-flash')])
12345678


In [3]:
from client.LLMclientgeneric import LLMclientgeneric
def KG_tool(
    question: str,
    history: List[List | None] = None,
):
    kg_info = None
    try:
        # 此处在使用知识图谱之前，需先检查问题的实体
        entities = check_entity(question)
        kg_info = relation_tool(entities)
    except:
        pass
        
    if kg_info is not None:
        print(f"KG_tool: \n {kg_info}")
        question = f"{question}\n从知识图谱中检索到的信息如下{kg_info}\n请你基于知识图谱的信息去回答,并给出知识图谱检索到的信息"

    response = LLMclientgeneric().chat_with_ai_stream(question, history)
    return response

In [4]:
answer = KG_tool(question='介绍糖尿病')
print(answer)

KG_tool: 
 广谱抗生素 好评药品 糖尿病；糖尿病 治疗方法 支持性治疗；腓肠神经活检 诊断检查 糖尿病；急性女阴溃疡 并发症 糖尿病；糖尿 症状 CNS；脑血管损害 并发症 糖尿病；急性胰腺炎 并发症 糖尿病；血管性帕金森综合征 并发症 糖尿病；卡他莫拉菌感染 症状 糖尿；糖尿病 并发症 糖尿病乳酸性酸中毒；精神障碍 并发症 糖尿病；白癜风 并发症 糖尿病；胰腺脑病综合征 症状 糖尿；糖尿病 症状 烦渴多饮；珠蛋白生成障碍性贫血 并发症 糖尿病；胰痈 并发症 糖尿病；贝特类 好评药品 糖尿病；糖尿病 好评药品 天麦消渴片；阿司匹林 好评药品 糖尿病；心烦 症状 糖尿病；红斑性肢痛症 并发症 糖尿病；糖尿病足 症状 糖尿；糖尿病 常用药品 降糖宁胶囊；低血糖 并发症 糖尿病；自身免疫性胰腺炎 并发症 糖尿病；遗传性肥胖 症状 糖尿；糖尿病 并发症 HIV相关呼吸道感染；胃动过速 症状 糖尿病；乏力 症状 糖尿病；ACE抑制剂 好评药品 糖尿病；反复眩晕发作 症状 糖尿病；老年肺炎 并发症 糖尿病；糖尿病 预防方法: 日常预防
1、一级预防：树立正确的进食观，采取合理的生活方式。糖尿病虽存在一定的遗传因素，但关键是生活因素和环境因素。热量过度摄入、营养过剩、肥胖、缺少运动是发病的重要原因。热量摄入适当，低盐、低糖、低脂、高纤维、维生素充足，是最佳的饮食配伍。
2、二级预防：定期测量血糖，以尽早发现无症状性糖尿病。应该将血糖测定列入中老年常规的体检项目，即使一次正常者，仍要定期测定。凡有糖尿病蛛丝马迹可寻者，如有皮肤感觉异常、性功能减退、视力不佳、多尿、白内障等，更要及时去测定和仔细鉴别，以期尽早诊断，争得早期治疗的可贵时间。
3、三级预防：糖尿病人很容易并发其他慢性病，患者多因并发症而危及生命。因此，要对糖尿病慢性合并症加强监测，做到早期发现，早期预防，而到了晚期，疗效往往不佳。早期诊断和早期治疗，常可预防并发症的发生，使病人能长期过上接近正常人的生活。
健康教育
1、由于约有半数以上的早期患者并无症状或症状轻微，常不能及时确诊和得到防治，因而要大力开展糖尿病宣传健康教育，让已确诊的患者了解糖尿病并逐渐熟悉饮食，运动，用药和尿糖、血糖监测等基本措施的综合治疗原则。
2、配合医务人员提高控制质量，让>50岁的对象，尤其是前述高危对象，每年做一次餐后2小时的血糖筛

In [5]:
type(answer)

openai.Stream

In [6]:
history = []
bot_response = ""
for chunk in answer:
    bot_response = bot_response + (chunk.choices[0].delta.content or "")
    if chunk.choices[0].finish_reason == "stop":
        break
    history.append(chunk)
print(bot_response)
type(bot_response)

糖尿病是一种常见的内分泌代谢性疾病，其发病的主要原因是胰岛素分泌不足以及胰升高血糖素分泌过多。这种疾病多见于40岁以上的肥胖人群，且常有家族史，因此与遗传因素密切相关。部分病例可能与病毒感染和自身免疫反应有关。

### 症状：
糖尿病的主要症状包括烦渴、多饮、多尿、多食、乏力、消瘦等。此外，还可能出现尿糖升高、血糖值升高等现象。

### 病因：
- **Ⅰ型糖尿病**：存在免疫系统异常，在某些病毒感染（如柯萨奇病毒、风疹病毒等）后引发自身免疫反应，破坏胰岛素β细胞。
- **Ⅱ型糖尿病**：主要由遗传易感性和环境因素共同作用导致，例如进食过多、体力活动减少引起的肥胖。

### 并发症：
糖尿病可以引发多种并发症，包括但不限于：
- 大血管和微血管病变
- 心肌梗死
- 肾病
- 神经病变
- 视网膜病变
- 高血压
- 高脂血症

### 治疗方法：
1. **药物治疗**：常用的药物包括盐酸二甲双胍片、消渴平片、十八味诃子利尿胶囊等。
2. **生活方式调整**：包括饮食控制、适量运动及情绪调节。
3. **中医治疗**：采用疏肝调气的方法来改善症状。

### 预防方法：
1. **一级预防**：树立正确的进食观念，采取健康的生活方式。
2. **二级预防**：定期测量血糖，尽早发现无症状性糖尿病。
3. **三级预防**：加强对慢性并发症的监测，做到早发现、早治疗。

### 好评药品：
一些被推荐用于治疗糖尿病的药物包括：
- 盐酸二甲双胍缓释片
- 渴乐宁胶囊
- 天麦消渴片
- 利拉鲁肽

以上信息来源于知识图谱中的相关内容。


str